In [27]:

from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 12*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------
# tf.config.set_visible_devices([], 'GPU') #run on CPU only -- seems to actually execute main parts of code faster here...

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [111]:
# test- raw Pixell data?
prefix = "/media/derm/06EF-127D1/leddartech/20200721_144638_part36_1956_2229/pixell_bfc_ech/" #test
fn1 = prefix + "00000150.pkl"
with open(fn1, 'rb') as f:
    data = pickle.load(f)
# print(data)
data = data['data'] #just want to hold on to Pixell measurements
# print(np.shape(data))

In [122]:
#Ouster OS1-64 mechanical LiDAR
prefix = "/home/derm/Downloads/20200721_144638_part36_1956_2229/ouster64_bfc_xyzit/" #very good quality
fn1 = prefix + "00000025.pkl"
with open(fn1, 'rb') as f:
    data = pickle.load(f)

data = np.asarray(data.tolist())[:,:3]
# print(np.shape(data))
# print(data[:10])

In [121]:
plt1 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp = []
disp.append(Points(data, c = 'red', r = 2.5, alpha = 1))
plt1.show(disp, "Fig LedderTech")
ViewInteractiveWidget(plt1.window)

ViewInteractiveWidget(height=1598, layout=Layout(height='auto', width='100%'), width=1280)